In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [11]:
# Path of the file to read
titanic_test_path = '../input/titanic/test.csv'
titanic_file_path = '../input/titanic/train.csv'

#Put csv files into a dataframe
titanicTestData = pd.read_csv(titanic_test_path)
titanicTrainData = pd.read_csv(titanic_file_path)
titanicTrainData.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [12]:
import itertools

#setup target variable and features for training and testing data
y = titanicTrainData.Survived

features = ['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch']
test_X = titanicTestData[features]
X = titanicTrainData[features]

#Apply preprocessing for features
def genderNum(gender):
    if gender == 'male':
        return 0
    else:
        return 1

#change genders from strings to integer values
X.loc[:,'Sex'] = X.Sex.apply(genderNum)
test_X.loc[:,'Sex'] = test_X.Sex.apply(genderNum)

#fill null age values with means of people grouped by surviving, sex, and class
X['Survived'] = y
groupTrainMeans = X.groupby(['Survived', 'Sex', 'Pclass'])['Age'].agg('mean').round(0)
print(groupTrainMeans)
for i, j, k in itertools.product(range(2), range(2), range(1,4)):
    X.loc[:,'Age'] = np.where(pd.isnull(X.Age) &
                              np.logical_and(X.Survived == i,
                                             np.logical_and(X.Pclass == k, X.Sex == j)),
                                             groupTrainMeans[i][j][k], X.Age)

#delete the survived column after using it for grouping
del X['Survived']

#fill null age values of test data  with means of people grouped by sex and class (we do not have
#the data if they survived or not so thats why it has been ignored)
groupTestMeans = test_X.groupby(['Sex', 'Pclass'])['Age'].agg('mean').round(0)
print(groupTestMeans)
for i, j in itertools.product(range(2), range(1,4)):
    test_X.loc[:,'Age'] = np.where(pd.isnull(test_X.Age) 
                                   & np.logical_and(test_X.Pclass == j, test_X.Sex == i),
                                   groupTestMeans[i][j], test_X.Age)

#fill in missing fare values with mean fare value
test_X.Fare.fillna(test_X.Fare.mean(), inplace = True)

Survived  Sex  Pclass
0         0    1         45.0
               2         33.0
               3         27.0
          1    1         26.0
               2         36.0
               3         24.0
1         0    1         36.0
               2         16.0
               3         22.0
          1    1         35.0
               2         28.0
               3         19.0
Name: Age, dtype: float64
Sex  Pclass
0    1         41.0
     2         31.0
     3         25.0
1    1         41.0
     2         24.0
     3         23.0
Name: Age, dtype: float64


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

#split training data to have testing data available
def printScore(estimators, depth):
    titanic_model = RandomForestClassifier(random_state=1, n_estimators=estimators, max_depth=depth)
    scores = cross_val_score(titanic_model, X, y, cv=10)
    print('predictions for', estimators, 'estimators and', depth, 'as max depth')
    print(np.mean(scores))
    return np.mean(scores)

#best score in terms of accuracy to identify which parameters will be chosen
#we give the score an initial value before entering the for loop so we can have a benchmark to start with
bestScore = printScore(5, 5)
best_estimator = 5
best_depth = 5
candidate_estimators = [5, 10, 25, 100, 250, 500]
candidate_max_depth = [5, 10, 15, 20, 25]
for i in candidate_estimators:
    for j in candidate_max_depth:
        if i == 5 and j == 5:
            continue

        current_loopScore = printScore(i, j)
        if current_loopScore > bestScore:
            bestScore = current_loopScore
            best_estimator = i
            best_depth = j

titanic_model = RandomForestClassifier(random_state=1, n_estimators=best_estimator, max_depth=best_depth)
titanic_model.fit(X,y)
predVals = titanic_model.predict(test_X)

predictionDf = pd.DataFrame({'PassengerId': titanicTestData.PassengerId, 'Survived': predVals})
print(predictionDf.head())
print(predictionDf.describe())

predictionDf.to_csv('../working/predictions.csv', index = False)

predictions for 5 estimators and 5 as max depth
0.8440823970037453
predictions for 5 estimators and 10 as max depth
0.8362421972534332
predictions for 5 estimators and 15 as max depth
0.8204744069912608
predictions for 5 estimators and 20 as max depth
0.8182397003745319
predictions for 5 estimators and 25 as max depth
0.8182397003745319
predictions for 10 estimators and 5 as max depth
0.8373033707865168
predictions for 10 estimators and 10 as max depth
0.8328464419475654
predictions for 10 estimators and 15 as max depth
0.821573033707865
predictions for 10 estimators and 20 as max depth
0.8294756554307117
predictions for 10 estimators and 25 as max depth
0.8272284644194757
predictions for 25 estimators and 5 as max depth
0.8373033707865168
predictions for 25 estimators and 10 as max depth
0.8440699126092385
predictions for 25 estimators and 15 as max depth
0.8260923845193509
predictions for 25 estimators and 20 as max depth
0.8350561797752809
predictions for 25 estimators and 25 as max